In [1]:
from tree_sitter import Language

Language.build_library(
    # Store the library in the `build` directory
    "build/my-languages.so",
    # Include one or more languages
    ["vendor/tree-sitter-go", "vendor/tree-sitter-javascript", "vendor/tree-sitter-python"],
)

False

In [2]:
GO_LANGUAGE = Language("build/my-languages.so", "go")
JS_LANGUAGE = Language("build/my-languages.so", "javascript")
PY_LANGUAGE = Language("build/my-languages.so", "python")

In [3]:
from tree_sitter import Parser

parser = Parser()
parser.set_language(PY_LANGUAGE)

In [17]:
tree = parser.parse(
    bytes(
        """
def foo():
    if bar:
        baz()
""",
        "utf8",
    )
)

In [ ]:
root_node = tree.root_node
print(root_node.text)
assert root_node.type == 'module'
print("root_node:",root_node)
# assert root_node.start_point == (1, 0)
# assert root_node.end_point == (3, 13)

function_node = root_node.children[0]
print("function_node:",function_node)
assert function_node.type == 'function_definition'
print("debug",function_node.children[1].type,function_node.children[1].text,function_node.children[1].children)
assert function_node.child_by_field_name('name').type == 'identifier'

function_name_node = function_node.children[1]
print("function_name_node",function_name_node)
assert function_name_node.type == 'identifier'
assert function_name_node.start_point == (1, 4)
assert function_name_node.end_point == (1, 7)

print(root_node.sexp())

In [ ]:
cursor = tree.walk()

assert cursor.node.type == "module"

assert cursor.goto_first_child()
assert cursor.node.type == "function_definition"

assert cursor.goto_first_child()
assert cursor.node.type == "def"

# Returns `False` because the `def` node has no children
assert not cursor.goto_first_child()

assert cursor.goto_next_sibling()
assert cursor.node.type == "identifier"

assert cursor.goto_next_sibling()
assert cursor.node.type == "parameters"

assert cursor.goto_parent()
assert cursor.node.type == "function_definition"

In [25]:
query = PY_LANGUAGE.query(
    """
(function_definition
  name: (identifier) @function.def)

(call
  function: (identifier) @function.call)
"""
)

captures = query.captures(tree.root_node)
assert len(captures) == 2
print(captures[0][0].text)
# assert captures[0][0] == "function_name_node"
assert captures[0][1] == "function.def"

b'foo'


In [16]:
print(tree.text)

tree.edit(
    start_byte=5,
    old_end_byte=5,
    new_end_byte=5 + 2,
    start_point=(0, 5),
    old_end_point=(0, 5),
    new_end_point=(0, 5 + 2),
)

print(tree.text)

b'\ndef foo():\n    if bar:\n        baz()\n'
None


In [9]:

new_tree = parser.parse(bytes(tree.text), old_tree=tree)
print(new_tree.text)


None


TypeError: cannot convert 'NoneType' object to bytes